In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.13.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [5]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0 #1로맞추기
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [6]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [7]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate) :
    return tf.keras.layers.Dropout(rate)

In [8]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())
            self.model.add(dropout(rate=0.5))

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [9]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())
        model.add(dropout(rate=0.5))

    model.add(dense(label_dim, weight_init))

    return model

In [10]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))


In [11]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [12]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\knh11\anaconda3\envs\tfpy3.10\lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 1.5893, train_loss: 2.26315665, train_accuracy: 0.2188, test_Accuracy: 0.1381
Epoch: [ 0] [    1/  468] time: 2.1519, train_loss: 2.36597919, train_accuracy: 0.1250, test_Accuracy: 0.1340
Epoch: [ 0] [    2/  468] time: 2.6972, train_loss: 2.32256174, train_accuracy: 0.1250, test_Accuracy: 0.1204
Epoch: [ 0] [    3/  468] time: 3.2570, train_loss: 2.27502108, train_accuracy: 0.1250, test_Accuracy: 0.1218
Epoch: [ 0] [    4/  468] time: 4.1172, train_loss: 2.31808233, train_accuracy: 0.1016, test_Accuracy: 0.1771
Epoch: [ 0] [    5/  468] time: 4.8723, train_loss: 2.21843219, train_accuracy: 0.3359, test_Accuracy: 0.2872
Epoch: [ 0] [    6/  468] time: 5.7927, train_loss: 2.25531721, train_accuracy: 0.4062, test_Accuracy: 0.4201
Epoch: [ 0] [    7/  468] time: 6.5234, train_loss: 2.22985029, train_accuracy: 0.4375, test_Accuracy: 0.4864
Epoch: [ 0] [    8/  468] time: 7.1898, train_loss: 2.18543959, train_accuracy: 0.5469, test_Accuracy: 0.5233
Epoch: [ 0

Epoch: [ 0] [   74/  468] time: 54.3743, train_loss: 0.68193501, train_accuracy: 0.8516, test_Accuracy: 0.8691
Epoch: [ 0] [   75/  468] time: 55.0691, train_loss: 0.55345106, train_accuracy: 0.8906, test_Accuracy: 0.8703
Epoch: [ 0] [   76/  468] time: 55.7339, train_loss: 0.51999998, train_accuracy: 0.8750, test_Accuracy: 0.8770
Epoch: [ 0] [   77/  468] time: 56.3858, train_loss: 0.61858904, train_accuracy: 0.8438, test_Accuracy: 0.8797
Epoch: [ 0] [   78/  468] time: 57.1085, train_loss: 0.58115697, train_accuracy: 0.9219, test_Accuracy: 0.8797
Epoch: [ 0] [   79/  468] time: 57.9083, train_loss: 0.56054378, train_accuracy: 0.8672, test_Accuracy: 0.8813
Epoch: [ 0] [   80/  468] time: 58.5457, train_loss: 0.70644522, train_accuracy: 0.8750, test_Accuracy: 0.8872
Epoch: [ 0] [   81/  468] time: 59.2091, train_loss: 0.61648226, train_accuracy: 0.9141, test_Accuracy: 0.8916
Epoch: [ 0] [   82/  468] time: 60.0099, train_loss: 0.58005381, train_accuracy: 0.9297, test_Accuracy: 0.8949
E

Epoch: [ 0] [  148/  468] time: 107.1801, train_loss: 0.39626950, train_accuracy: 0.9141, test_Accuracy: 0.9251
Epoch: [ 0] [  149/  468] time: 107.8232, train_loss: 0.34277827, train_accuracy: 0.9219, test_Accuracy: 0.9244
Epoch: [ 0] [  150/  468] time: 108.5624, train_loss: 0.41677091, train_accuracy: 0.8984, test_Accuracy: 0.9243
Epoch: [ 0] [  151/  468] time: 109.2202, train_loss: 0.33386940, train_accuracy: 0.9609, test_Accuracy: 0.9232
Epoch: [ 0] [  152/  468] time: 109.8434, train_loss: 0.30839041, train_accuracy: 0.9141, test_Accuracy: 0.9225
Epoch: [ 0] [  153/  468] time: 110.6983, train_loss: 0.47003949, train_accuracy: 0.8984, test_Accuracy: 0.9218
Epoch: [ 0] [  154/  468] time: 111.3350, train_loss: 0.34574667, train_accuracy: 0.9297, test_Accuracy: 0.9236
Epoch: [ 0] [  155/  468] time: 111.9054, train_loss: 0.50103050, train_accuracy: 0.8984, test_Accuracy: 0.9242
Epoch: [ 0] [  156/  468] time: 112.6575, train_loss: 0.48980114, train_accuracy: 0.9141, test_Accuracy:

Epoch: [ 0] [  222/  468] time: 161.3936, train_loss: 0.17060558, train_accuracy: 0.9531, test_Accuracy: 0.9341
Epoch: [ 0] [  223/  468] time: 162.1119, train_loss: 0.26503801, train_accuracy: 0.9531, test_Accuracy: 0.9344
Epoch: [ 0] [  224/  468] time: 162.7623, train_loss: 0.39669555, train_accuracy: 0.8984, test_Accuracy: 0.9345
Epoch: [ 0] [  225/  468] time: 163.4512, train_loss: 0.33357221, train_accuracy: 0.9375, test_Accuracy: 0.9368
Epoch: [ 0] [  226/  468] time: 164.1994, train_loss: 0.29297897, train_accuracy: 0.9453, test_Accuracy: 0.9377
Epoch: [ 0] [  227/  468] time: 164.9992, train_loss: 0.26063958, train_accuracy: 0.9609, test_Accuracy: 0.9378
Epoch: [ 0] [  228/  468] time: 165.8926, train_loss: 0.40856802, train_accuracy: 0.9062, test_Accuracy: 0.9367
Epoch: [ 0] [  229/  468] time: 166.5241, train_loss: 0.42204607, train_accuracy: 0.9453, test_Accuracy: 0.9332
Epoch: [ 0] [  230/  468] time: 167.2684, train_loss: 0.37965971, train_accuracy: 0.9531, test_Accuracy:

Epoch: [ 0] [  296/  468] time: 215.9676, train_loss: 0.34238225, train_accuracy: 0.9375, test_Accuracy: 0.9447
Epoch: [ 0] [  297/  468] time: 216.6931, train_loss: 0.35892549, train_accuracy: 0.9609, test_Accuracy: 0.9416
Epoch: [ 0] [  298/  468] time: 217.3307, train_loss: 0.35145849, train_accuracy: 0.9219, test_Accuracy: 0.9390
Epoch: [ 0] [  299/  468] time: 218.1248, train_loss: 0.34058532, train_accuracy: 0.9297, test_Accuracy: 0.9390
Epoch: [ 0] [  300/  468] time: 218.8115, train_loss: 0.35645431, train_accuracy: 0.9531, test_Accuracy: 0.9413
Epoch: [ 0] [  301/  468] time: 219.4279, train_loss: 0.37056240, train_accuracy: 0.9375, test_Accuracy: 0.9429
Epoch: [ 0] [  302/  468] time: 220.2280, train_loss: 0.24956186, train_accuracy: 0.9688, test_Accuracy: 0.9445
Epoch: [ 0] [  303/  468] time: 221.0200, train_loss: 0.19157603, train_accuracy: 0.9766, test_Accuracy: 0.9449
Epoch: [ 0] [  304/  468] time: 221.7786, train_loss: 0.17845993, train_accuracy: 0.9766, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 270.9732, train_loss: 0.24262093, train_accuracy: 0.9688, test_Accuracy: 0.9478
Epoch: [ 0] [  371/  468] time: 271.7253, train_loss: 0.25432295, train_accuracy: 0.9766, test_Accuracy: 0.9499
Epoch: [ 0] [  372/  468] time: 272.4042, train_loss: 0.34067807, train_accuracy: 0.9297, test_Accuracy: 0.9508
Epoch: [ 0] [  373/  468] time: 272.9919, train_loss: 0.26997107, train_accuracy: 0.9609, test_Accuracy: 0.9509
Epoch: [ 0] [  374/  468] time: 273.7510, train_loss: 0.33066267, train_accuracy: 0.9531, test_Accuracy: 0.9514
Epoch: [ 0] [  375/  468] time: 274.5524, train_loss: 0.23009443, train_accuracy: 0.9453, test_Accuracy: 0.9512
Epoch: [ 0] [  376/  468] time: 275.4046, train_loss: 0.29199460, train_accuracy: 0.9219, test_Accuracy: 0.9511
Epoch: [ 0] [  377/  468] time: 276.1210, train_loss: 0.35647434, train_accuracy: 0.9375, test_Accuracy: 0.9503
Epoch: [ 0] [  378/  468] time: 276.9026, train_loss: 0.38776654, train_accuracy: 0.9297, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 323.5279, train_loss: 0.25023481, train_accuracy: 0.9453, test_Accuracy: 0.9545
Epoch: [ 0] [  445/  468] time: 324.3509, train_loss: 0.14664039, train_accuracy: 0.9766, test_Accuracy: 0.9543
Epoch: [ 0] [  446/  468] time: 325.0608, train_loss: 0.34303290, train_accuracy: 0.9531, test_Accuracy: 0.9545
Epoch: [ 0] [  447/  468] time: 325.7750, train_loss: 0.23622678, train_accuracy: 0.9609, test_Accuracy: 0.9545
Epoch: [ 0] [  448/  468] time: 326.5608, train_loss: 0.31755394, train_accuracy: 0.9766, test_Accuracy: 0.9542
Epoch: [ 0] [  449/  468] time: 327.3514, train_loss: 0.34188044, train_accuracy: 0.9375, test_Accuracy: 0.9543
Epoch: [ 0] [  450/  468] time: 328.0905, train_loss: 0.21967778, train_accuracy: 0.9531, test_Accuracy: 0.9532
Epoch: [ 0] [  451/  468] time: 328.7909, train_loss: 0.42210090, train_accuracy: 0.9375, test_Accuracy: 0.9538
Epoch: [ 0] [  452/  468] time: 329.3626, train_loss: 0.26754677, train_accuracy: 0.9453, test_Accuracy:

# Test accuracy: 95.34%